In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/TED_for_bert.pickle
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/NgDGlcxYrhQ.txt
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/MvXZzKZ3JYQ.txt
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/NDQ1Mi5I4rg.txt
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/yFVXsjVdvmY.txt
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/jAemh_JxgOk.txt
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/xlUPlxSpDRo.txt
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/WfTZ5iIUn4s.txt
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/os69gEFXUW4.txt
/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/TED/TED/subtitle_set/XYNUp

> **ENVIRONMENT SETUP**

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Installing all dependencies

In [3]:
print("Installing all dependencies")
!pip install -q transformers[torch] "datasets[squad]" sentencepiece pytube openai-whisper graphviz evaluate rouge_score
# Install system dependency (ffmpeg)
!apt-get -y install ffmpeg > /dev/null 2>&1
print("All dependencies are installed.")

Installing all dependencies
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
print("\n Importing libraries and setting configuration")
import torch
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

MODEL_CHECKPOINT = "t5-base"
OUTPUT_DIR = "./t5-finetuned-tech-talks"


 Importing libraries and setting configuration


2025-07-27 05:40:34.489854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753594834.715557      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753594834.777300      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Dataset Loading

In [5]:
print("\n Loading and preparing the dataset")

def load_tech_talk_sum_dataset(dataset_path):
    """Loads transcripts and summaries from the tech-talk-sum directory structure."""
    data = []
    channels = ["DeepLearningTV", "MicrosoftResearch", "TED", "TechInsider", "TensorFlow", "TwoMinutePaper"]
    
    if not os.path.exists(dataset_path):
        raise FileNotFoundError(f"Dataset path not found: {dataset_path}. Please check the folder name in /kaggle/input/.")

    for channel in channels:
        channel_path = os.path.join(dataset_path, channel, channel) 
        subtitle_dir = os.path.join(channel_path, "subtitle_set")
        summary_dir = os.path.join(channel_path, "summary_set")
        
        if os.path.isdir(subtitle_dir):
            for filename in os.listdir(subtitle_dir):
                if filename.endswith(".txt"):
                    transcript_path = os.path.join(subtitle_dir, filename)
                    summary_path = os.path.join(summary_dir, filename)
                    if os.path.exists(summary_path):
                        try:
                            with open(transcript_path, 'r', encoding='utf-8') as f_text, open(summary_path, 'r', encoding='utf-8') as f_summary:
                                data.append({"text": f_text.read(), "summary": f_summary.read()})
                        except Exception as e:
                            print(f"Skipping file {filename} due to error: {e}")
    return data

DATASET_BASE_PATH = "/kaggle/input/tech-talk-sum-dataset/Tech-Talk_Sum_dataset-master/"
raw_data = load_tech_talk_sum_dataset(DATASET_BASE_PATH)
full_dataset = Dataset.from_list(raw_data)
dataset = full_dataset.train_test_split(test_size=0.1, seed=42) 
print(f"Dataset loaded successfully: {len(dataset['train'])} training examples, {len(dataset['test'])} test examples.")


 Loading and preparing the dataset
Dataset loaded successfully: 453 training examples, 51 test examples.


# Testing Pre-Trained Model

In [6]:
print("\n Testing the PRE-TRAINED T5 model")

base_summarizer = pipeline("text2text-generation", model=MODEL_CHECKPOINT, device=0)
sample = dataset['test'][0]
reference_summary = sample['summary']
article_text = sample['text']

base_model_summary = base_summarizer("summarize: " + article_text, max_length=128, min_length=30, truncation=True)[0]['generated_text']

print("\n COMPARISON")
print(f"ARTICLE (first 300 chars): {article_text[:300]}...")
print(f"\nREFERENCE SUMMARY: {reference_summary}")
print(f"\nPRE-TRAINED T5 MODEL SUMMARY: {base_model_summary}")


 Testing the PRE-TRAINED T5 model


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



 COMPARISON
ARTICLE (first 300 chars): I'd like to tell you about a legal case that I worked on involving a man named Steve Titus. Titus was a restaurant manager. He was 31 years old, he lived in Seattle, Washington, he was engaged to Gretchen, about to be married, she was the love of his life. And one night, the couple went out for a ro...

REFERENCE SUMMARY: Psychologist Elizabeth Loftus studies memories. More precisely, she studies false memories, when people either remember things that didn't happen or remember them differently from the way they really were. It's more common than you might think, and Loftus shares some startling stories and statistics, and raises some important ethical questions we should all remember to consider. 

PRE-TRAINED T5 MODEL SUMMARY: psychologist dr. eric liu says he's been sued for speaking out on memory issues . liu: we can't reliably distinguish true memories from false memories . liu: if we plant a false memory, it has repercussions that affect beh

# Fine-Tuning Model

In [7]:
print("\n Setting up for fine-tuning")

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
prefix = "summarize: "

def preprocess_function(examples):
    inputs = tokenizer([prefix + doc for doc in examples["text"]], max_length=1024, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
print("Data tokenized for T5.")

rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}
print("Metrics function defined.")


 Setting up for fine-tuning


Map:   0%|          | 0/453 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Data tokenized for T5.


Metrics function defined.


In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    predict_with_generate=True,
    report_to="none",  
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
print("Fine-tuning completed")

trainer.save_model(OUTPUT_DIR)
print(f" Best T5 model saved to {OUTPUT_DIR}")

/tmp/ipykernel_36/2294044252.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,4.975400,3.247593,0.137000,0.024200,0.103800,0.104000
2,2.463300,2.259079,0.000000,0.000000,0.000000,0.000000
3,2.104600,2.168088,0.141900,0.038800,0.109500,0.109200


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Fine-tuning completed
 Best T5 model saved to ./t5-finetuned-tech-talks


# Testing Fine-Tuned Model

In [11]:
print("\n Testing the final FINE-TUNED T5 model")

finetuned_summarizer = pipeline("text2text-generation", model=OUTPUT_DIR, device=0)

model_summary = finetuned_summarizer("summarize: " + article_text, max_length=128, min_length=30, truncation=True)[0]['generated_text']

print("\n FINAL COMPARISON ON TEST DATA")
print(f"ARTICLE (first 300 chars): {article_text[:300]}...")
print(f"\nREFERENCE SUMMARY: {reference_summary}")
print(f"\nFINE-TUNED T5 MODEL SUMMARY: {model_summary}")


 Testing the final FINE-TUNED T5 model


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



 FINAL COMPARISON ON TEST DATA
ARTICLE (first 300 chars): I'd like to tell you about a legal case that I worked on involving a man named Steve Titus. Titus was a restaurant manager. He was 31 years old, he lived in Seattle, Washington, he was engaged to Gretchen, about to be married, she was the love of his life. And one night, the couple went out for a ro...

REFERENCE SUMMARY: Psychologist Elizabeth Loftus studies memories. More precisely, she studies false memories, when people either remember things that didn't happen or remember them differently from the way they really were. It's more common than you might think, and Loftus shares some startling stories and statistics, and raises some important ethical questions we should all remember to consider. 

FINE-TUNED T5 MODEL SUMMARY: In a disturbing trend in the United States, researchers are being sued for simply speaking out on matters of great public controversy. In this disturbing trend, researchers are being sued for simply speak